In [10]:
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/ship.csv

--2020-05-10 19:02:27--  https://github.com/mlittmancs/great_courses_ml/raw/master/ship.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/ship.csv [following]
--2020-05-10 19:02:27--  https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/ship.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61194 (60K) [text/plain]
Saving to: ‘ship.csv’

ship.csv            100%[===================>]  59.76K  --.-KB/s    in 0.02s   

2020-05-10 19:02:27 (2.41 MB/s) - ‘ship.csv’ saved [61194/61194]



In [0]:
import csv
   
first = True
with open("ship.csv") as f:
    csvdata = csv.reader(f, delimiter=',')
    data = []
    for row in csvdata:
      if not first: data += [row]
      first = False

array = []
for col in range(len(data[0])):
  array += [{}]
  new = 0
  for i in range(len(data)):	
     line = data[i]		
     if line[col] not in array[col]:
      array[col][line[col]] = new
      new += 1  
alldat = []
alllabs = []
for line in data:
  alllabs += [int(line[1])]
  if line[5] == '': line[5] = '50'
  alldat += [ [int(line[2]), array[4][line[4]], float(line[5]), int(line[6]), int(line[7]), float(line[9]), line[11]=='S', line[11]=='C', line[11]=='Q' ]]
feats = ['Pclass','Sex','Age','SibSp','Parch','Fare', 'Embarked S', 'Embarked C', 'Embarked Q']

In [0]:
import random

trainmask = [random.randint(0,2) for i in range(len(alldat))]

traindat = [alldat[i] for i in range(len(alldat)) if trainmask[i]<2]
trainlabs = [alllabs[i] for i in range(len(alldat)) if trainmask[i]<2]
testdat = [alldat[i] for i in range(len(alldat)) if trainmask[i]==2]
testlabs = [alllabs[i] for i in range(len(alldat)) if trainmask[i]==2]


In [23]:
from sklearn.neural_network import MLPClassifier

nhidden = 60
clf = MLPClassifier(hidden_layer_sizes=[nhidden], max_iter = 50000)
clf = clf.fit(traindat, trainlabs)
pred = clf.predict(testdat)
[sum([pred[i] != testlabs[i] for i in range(len(testlabs))]) / len(testlabs)]


[0.16831683168316833]

In [26]:
# feats = ['Pclass','Sex','Age','SibSp','Parch','Fare', 'Embarked S', 'Embarked C', 'Embarked Q']

imp = []
for v in alldat:
  real = v[1]
  v[1] = 0
  asmale = clf.predict_proba([v])[0][1]
  v[1] = 1
  asfemale = clf.predict_proba([v])[0][1]
  v[1] = real
  imp += [ asfemale-asmale ]
  
print(sum(imp)/len(imp))

0.43800593780752317


In [31]:

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter = 500)

clf.fit(traindat, trainlabs)  

pred = clf.predict(traindat)
trainerr = sum([pred[i] != trainlabs[i] for i in range(len(trainlabs))]) / len(trainlabs)
pred = clf.predict(testdat)
testerr = sum([pred[i] != testlabs[i] for i in range(len(testlabs))]) / len(testlabs)

print(trainerr, testerr)

0.20408163265306123 0.18151815181518152


In [32]:
for i in range(len(feats)):
  print(feats[i], clf.coef_[0][i])


Pclass -1.0842804487221074
Sex 2.3727578593518097
Age -0.029319924819888245
SibSp -0.3781468800259447
Parch -0.002807055755946123
Fare -0.0030156176553032504
Embarked S -0.4910865643071035
Embarked C 0.1615775858739615
Embarked Q 0.33086016494274
